# Natural language processing wit spacy

Gracias al procesamiento del lenguaje natural (PLN), voy a extraer a partir de un texto triple (sujeto, objeto, relación) para identificar tanto los nodos como las relaciones que incluirá el grafo de conocimineto.

## Librerias

Existen diferentes librerías en Python para PLN, entre las más conocidas estan NLTK y spaCy. Para este trabajo se usará spaCy (https://spacy.io/), la última versión que hay a 7 de diciembre de 2021.

In [2]:
!pip install -U pip setuptools wheel
!pip install -U spacy


In [2]:
import spacy
python -m spacy download en_core_web_sm

print(spacy.__version__)

3.0.3
